In [ ]:
# 📦 필수 라이브러리 Import
import sys
import os
from pathlib import Path
import warnings

# 프로젝트 루트 경로를 Python path에 추가
project_root = Path().absolute()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# 경고 메시지 필터링 (깨끗한 출력을 위해)
warnings.filterwarnings('ignore')

# 데이터 처리 및 분석
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 날짜 처리
from datetime import datetime, timedelta

# 프로젝트 모듈
try:
    from src.data_generation.data_generator import DataGenerator
    from src.utils.config_loader import load_config
    from src.utils.logger import get_logger
    print("✅ 프로젝트 모듈 import 성공")
except ImportError as e:
    print(f"⚠️  프로젝트 모듈 import 실패: {e}")
    print("현재 작업 디렉토리를 확인하고 프로젝트 루트에서 실행하세요.")

# 시각화 설정
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# pandas 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("🔧 환경 설정 완료!")
print(f"📍 현재 작업 디렉토리: {os.getcwd()}")
print(f"📊 pandas 버전: {pd.__version__}")
print(f"🔢 numpy 버전: {np.__version__}")


In [ ]:
# 🎛️ 데이터 생성 설정
config = {
    'n_customers': 1000,      # 고객 수
    'n_products': 100,        # 상품 수  
    'n_orders': 5000,         # 주문 수
    'start_date': '2023-01-01',   # 시작 날짜
    'end_date': '2024-12-31',     # 종료 날짜
    'seasonality_strength': 0.3,  # 계절성 강도
    'trend_strength': 0.2         # 트렌드 강도
}

print("📊 데이터 생성 설정:")
for key, value in config.items():
    print(f"  {key}: {value}")

# 🏭 데이터 생성기 초기화
print("\n🏭 데이터 생성기 초기화 중...")
generator = DataGenerator(config=config, random_seed=42)

print("\n✅ 데이터 생성기 준비 완료!")
print("⏰ 예상 소요 시간: 약 10-30초")


In [ ]:
# 🚀 데이터 생성 실행
print("🚀 데이터 생성을 시작합니다...\n")

# 모든 데이터 생성 (파일 저장 포함)
data = generator.generate_all(save_to_file=True)

# 생성된 데이터 확인
customers_df = data['customers']
products_df = data['products'] 
orders_df = data['orders']
order_items_df = data['order_items']

print("\n" + "="*50)
print("🎉 데이터 생성 완료!")
print("="*50)

# 생성된 데이터 요약
print(f"📊 생성된 데이터 요약:")
print(f"  👥 고객: {len(customers_df):,}명")
print(f"  📦 상품: {len(products_df):,}개")
print(f"  🛒 주문: {len(orders_df):,}건")
print(f"  📋 주문상품: {len(order_items_df):,}건")

# 파일 저장 위치 확인
data_dir = Path('data/raw')
if data_dir.exists():
    files = list(data_dir.glob('*.csv'))
    print(f"\n💾 저장된 파일 ({len(files)}개):")
    for file in files:
        file_size = file.stat().st_size / 1024  # KB
        print(f"  📄 {file.name}: {file_size:.1f} KB")
else:
    print("\n⚠️  data/raw 디렉토리를 찾을 수 없습니다.")


In [ ]:
# 👥 고객 데이터 기본 정보
print("👥 고객 데이터 기본 정보")
print("="*50)
print(f"📊 데이터 크기: {customers_df.shape[0]:,}행 × {customers_df.shape[1]}열")
print(f"💾 메모리 사용량: {customers_df.memory_usage(deep=True).sum() / 1024:.1f} KB")
print()

# 🔍 데이터 구조 확인
print("🔍 데이터 구조:")
print(customers_df.info())
print()

# 📋 첫 5행 미리보기
print("📋 데이터 미리보기 (첫 5행):")
print(customers_df.head())
print()

# 📈 기본 통계
print("📈 수치형 변수 기본 통계:")
print(customers_df.describe())


In [ ]:
# 👥 고객 데이터 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('👥 고객 데이터 분석', fontsize=16, fontweight='bold')

# 1️⃣ 나이 분포
axes[0, 0].hist(customers_df['age'], bins=20, color='skyblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('📊 나이 분포')
axes[0, 0].set_xlabel('나이')
axes[0, 0].set_ylabel('고객 수')
axes[0, 0].axvline(customers_df['age'].mean(), color='red', linestyle='--', 
                   label=f'평균: {customers_df["age"].mean():.1f}세')
axes[0, 0].legend()

# 2️⃣ 성별 분포
gender_counts = customers_df['gender'].value_counts()
axes[0, 1].pie(gender_counts.values, labels=['남성' if x=='M' else '여성' for x in gender_counts.index], 
               autopct='%1.1f%%', colors=['lightblue', 'lightpink'])
axes[0, 1].set_title('👫 성별 분포')

# 3️⃣ 지역 분포
region_counts = customers_df['region'].value_counts()
axes[1, 0].bar(region_counts.index, region_counts.values, color='lightgreen', alpha=0.7)
axes[1, 0].set_title('🗺️ 지역별 고객 분포')
axes[1, 0].set_xlabel('지역')
axes[1, 0].set_ylabel('고객 수')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4️⃣ 고객 세그먼트 분포
segment_counts = customers_df['segment'].value_counts()
colors = ['gold', 'silver', 'lightcoral', 'lightsteelblue']
axes[1, 1].bar(segment_counts.index, segment_counts.values, color=colors, alpha=0.7)
axes[1, 1].set_title('🎯 고객 세그먼트 분포')
axes[1, 1].set_xlabel('세그먼트')
axes[1, 1].set_ylabel('고객 수')

# 💄 레이아웃 조정
plt.tight_layout()
plt.show()

# 📊 범주형 변수 요약 통계
print("\n📊 범주형 변수 분포:")
print("\n👫 성별 분포:")
print(customers_df['gender'].value_counts())
print("\n🗺️ 지역 분포:")
print(customers_df['region'].value_counts())
print("\n🎯 세그먼트 분포:")
print(customers_df['segment'].value_counts())


In [ ]:
# 📦 상품 데이터 기본 정보
print("📦 상품 데이터 기본 정보")
print("="*50)
print(f"📊 데이터 크기: {products_df.shape[0]:,}행 × {products_df.shape[1]}열")
print()

# 📋 첫 5행 미리보기
print("📋 데이터 미리보기 (첫 5행):")
print(products_df.head())
print()

# 📈 가격 관련 기본 통계
print("📈 가격 관련 기본 통계:")
print(products_df[['price', 'discount_rate', 'final_price', 'stock']].describe())
print()

# 📊 범주형 변수 분포
print("📊 범주형 변수 분포:")
print("\n🏷️ 카테고리별 상품 수:")
print(products_df['category'].value_counts())
print("\n🏭 브랜드별 상품 수:")
print(products_df['brand'].value_counts())


In [ ]:
# 📦 상품 데이터 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📦 상품 데이터 분석', fontsize=16, fontweight='bold')

# 1️⃣ 최종 가격 분포
axes[0, 0].hist(products_df['final_price'], bins=20, color='lightcoral', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('💰 상품 가격 분포')
axes[0, 0].set_xlabel('가격 (원)')
axes[0, 0].set_ylabel('상품 수')
axes[0, 0].axvline(products_df['final_price'].mean(), color='red', linestyle='--',
                   label=f'평균: {products_df["final_price"].mean():,.0f}원')
axes[0, 0].legend()

# 2️⃣ 카테고리별 상품 수
category_counts = products_df['category'].value_counts()
axes[0, 1].bar(category_counts.index, category_counts.values, color='lightblue', alpha=0.7)
axes[0, 1].set_title('🏷️ 카테고리별 상품 수')
axes[0, 1].set_xlabel('카테고리')
axes[0, 1].set_ylabel('상품 수')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3️⃣ 할인율 분포
discount_counts = products_df['discount_rate'].value_counts().sort_index()
axes[1, 0].bar(discount_counts.index, discount_counts.values, color='gold', alpha=0.7)
axes[1, 0].set_title('🎯 할인율 분포')
axes[1, 0].set_xlabel('할인율 (%)')
axes[1, 0].set_ylabel('상품 수')

# 4️⃣ 카테고리별 평균 가격
avg_price_by_category = products_df.groupby('category')['final_price'].mean().sort_values(ascending=True)
axes[1, 1].barh(avg_price_by_category.index, avg_price_by_category.values, color='lightgreen', alpha=0.7)
axes[1, 1].set_title('🏷️ 카테고리별 평균 가격')
axes[1, 1].set_xlabel('평균 가격 (원)')
axes[1, 1].set_ylabel('카테고리')

# 💄 레이아웃 조정
plt.tight_layout()
plt.show()

# 💡 인사이트 출력
print("\n💡 상품 데이터 주요 인사이트:")
print(f"📊 가장 비싼 카테고리: {avg_price_by_category.idxmax()} ({avg_price_by_category.max():,.0f}원)")
print(f"📊 가장 저렴한 카테고리: {avg_price_by_category.idxmin()} ({avg_price_by_category.min():,.0f}원)")
print(f"🎯 할인 상품 비율: {(products_df['discount_rate'] > 0).mean() * 100:.1f}%")
print(f"💰 전체 평균 가격: {products_df['final_price'].mean():,.0f}원")


In [ ]:
# 🛒 주문 데이터 기본 정보
print("🛒 주문 데이터 기본 정보")
print("="*50)
print(f"📊 데이터 크기: {orders_df.shape[0]:,}행 × {orders_df.shape[1]}열")
print()

# 📋 첫 5행 미리보기
print("📋 데이터 미리보기 (첫 5행):")
print(orders_df.head())
print()

# 📈 주문 금액 관련 기본 통계
print("📈 주문 관련 기본 통계:")
print(orders_df[['total_amount', 'discount_amount', 'final_amount']].describe())
print()

# 📊 범주형 변수 분포
print("📊 범주형 변수 분포:")
print("\n📦 주문 상태별 분포:")
print(orders_df['status'].value_counts())
print("\n💳 결제 방법별 분포:")
print(orders_df['payment_method'].value_counts())

# 📅 날짜 관련 정보
print("\n📅 주문 날짜 범위:")
print(f"시작일: {orders_df['order_date'].min()}")
print(f"종료일: {orders_df['order_date'].max()}")
print(f"총 기간: {(orders_df['order_date'].max() - orders_df['order_date'].min()).days}일")


In [ ]:
# 🛒 주문 데이터 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('🛒 주문 데이터 분석', fontsize=16, fontweight='bold')

# 1️⃣ 주문 금액 분포
axes[0, 0].hist(orders_df['final_amount'], bins=20, color='lightgreen', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('💰 주문 금액 분포')
axes[0, 0].set_xlabel('주문 금액 (원)')
axes[0, 0].set_ylabel('주문 수')
axes[0, 0].axvline(orders_df['final_amount'].mean(), color='red', linestyle='--',
                   label=f'평균: {orders_df["final_amount"].mean():,.0f}원')
axes[0, 0].legend()

# 2️⃣ 주문 상태 분포
status_counts = orders_df['status'].value_counts()
colors = ['lightgreen', 'gold', 'lightcoral', 'lightblue']
axes[0, 1].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', colors=colors)
axes[0, 1].set_title('📦 주문 상태 분포')

# 3️⃣ 결제 방법별 분포
payment_counts = orders_df['payment_method'].value_counts()
axes[1, 0].bar(payment_counts.index, payment_counts.values, color='lightsalmon', alpha=0.7)
axes[1, 0].set_title('💳 결제 방법별 주문 분포')
axes[1, 0].set_xlabel('결제 방법')
axes[1, 0].set_ylabel('주문 수')

# 4️⃣ 일별 주문 건수 트렌드
daily_orders = orders_df.groupby(orders_df['order_date'].dt.date).size()
axes[1, 1].plot(daily_orders.index, daily_orders.values, color='steelblue', linewidth=2)
axes[1, 1].set_title('📈 일별 주문 건수 트렌드')
axes[1, 1].set_xlabel('날짜')
axes[1, 1].set_ylabel('주문 수')
axes[1, 1].tick_params(axis='x', rotation=45)

# 💄 레이아웃 조정
plt.tight_layout()
plt.show()

# 💡 주문 데이터 주요 인사이트
print("\n💡 주문 데이터 주요 인사이트:")
print(f"📊 총 주문 건수: {len(orders_df):,}건")
print(f"💰 평균 주문 금액: {orders_df['final_amount'].mean():,.0f}원")
print(f"💰 총 매출: {orders_df['final_amount'].sum():,.0f}원")
print(f"📦 완료된 주문 비율: {(orders_df['status'] == 'completed').mean() * 100:.1f}%")
print(f"🔄 반품/취소 비율: {((orders_df['status'] == 'returned') | (orders_df['status'] == 'cancelled')).mean() * 100:.1f}%")

# 📅 주문 패턴 분석
orders_df['weekday'] = orders_df['order_date'].dt.day_name()
print(f"📅 가장 주문이 많은 요일: {orders_df['weekday'].value_counts().index[0]}")
print(f"📅 가장 주문이 적은 요일: {orders_df['weekday'].value_counts().index[-1]}")


In [ ]:
# 📋 주문 상품 데이터 기본 정보
print("📋 주문 상품 데이터 기본 정보")
print("="*50)
print(f"📊 데이터 크기: {order_items_df.shape[0]:,}행 × {order_items_df.shape[1]}열")
print()

# 📋 첫 5행 미리보기
print("📋 데이터 미리보기 (첫 5행):")
print(order_items_df.head())
print()

# 📈 수량 및 가격 관련 통계
print("📈 수량 및 가격 관련 기본 통계:")
print(order_items_df[['quantity', 'unit_price', 'total_price']].describe())
print()

# 🔗 데이터 관계 확인
print("🔗 데이터 관계 확인:")
print(f"고유 주문 수: {order_items_df['order_id'].nunique():,}건")
print(f"고유 상품 수: {order_items_df['product_id'].nunique():,}개")
print(f"평균 주문당 상품 수: {len(order_items_df) / order_items_df['order_id'].nunique():.1f}개")

# 📊 인기 상품 TOP 10
print("\n📊 인기 상품 TOP 10 (주문 건수 기준):")
popular_products = order_items_df.groupby('product_id').agg({
    'quantity': 'sum',
    'total_price': 'sum'
}).sort_values('quantity', ascending=False).head(10)

# 상품명 매핑
popular_products_with_names = popular_products.merge(
    products_df[['product_id', 'product_name']], 
    left_index=True, 
    right_on='product_id'
).drop('product_id', axis=1)

print(popular_products_with_names)


In [ ]:
# 📋 주문 상품 데이터 시각화
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('📋 주문 상품 데이터 분석', fontsize=16, fontweight='bold')

# 1️⃣ 상품별 주문 수량 분포
axes[0, 0].hist(order_items_df['quantity'], bins=range(1, order_items_df['quantity'].max()+2), 
                color='skyblue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('📦 상품별 주문 수량 분포')
axes[0, 0].set_xlabel('수량')
axes[0, 0].set_ylabel('빈도')

# 2️⃣ 상품 단가 분포
axes[0, 1].hist(order_items_df['unit_price'], bins=20, color='lightcoral', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('💰 상품 단가 분포')
axes[0, 1].set_xlabel('단가 (원)')
axes[0, 1].set_ylabel('빈도')

# 3️⃣ 인기 상품 TOP 10 (수량 기준)
top_products_qty = order_items_df.groupby('product_id')['quantity'].sum().sort_values(ascending=False).head(10)
product_names = [products_df[products_df['product_id'] == pid]['product_name'].iloc[0][:15] + '...' 
                if len(products_df[products_df['product_id'] == pid]['product_name'].iloc[0]) > 15
                else products_df[products_df['product_id'] == pid]['product_name'].iloc[0]
                for pid in top_products_qty.index]

axes[1, 0].barh(range(len(top_products_qty)), top_products_qty.values, color='lightgreen', alpha=0.7)
axes[1, 0].set_yticks(range(len(top_products_qty)))
axes[1, 0].set_yticklabels(product_names)
axes[1, 0].set_title('🏆 인기 상품 TOP 10 (수량)')
axes[1, 0].set_xlabel('총 주문 수량')

# 4️⃣ 인기 상품 TOP 10 (매출 기준)
top_products_sales = order_items_df.groupby('product_id')['total_price'].sum().sort_values(ascending=False).head(10)
product_names_sales = [products_df[products_df['product_id'] == pid]['product_name'].iloc[0][:15] + '...' 
                      if len(products_df[products_df['product_id'] == pid]['product_name'].iloc[0]) > 15
                      else products_df[products_df['product_id'] == pid]['product_name'].iloc[0]
                      for pid in top_products_sales.index]

axes[1, 1].barh(range(len(top_products_sales)), top_products_sales.values/1000, color='gold', alpha=0.7)
axes[1, 1].set_yticks(range(len(top_products_sales)))
axes[1, 1].set_yticklabels(product_names_sales)
axes[1, 1].set_title('💎 인기 상품 TOP 10 (매출)')
axes[1, 1].set_xlabel('총 매출 (천원)')

# 💄 레이아웃 조정
plt.tight_layout()
plt.show()

# 💡 주문 상품 데이터 주요 인사이트
print("\n💡 주문 상품 데이터 주요 인사이트:")
print(f"📊 총 주문 상품 건수: {len(order_items_df):,}건")
print(f"📦 평균 주문 수량: {order_items_df['quantity'].mean():.1f}개")
print(f"💰 평균 상품 단가: {order_items_df['unit_price'].mean():,.0f}원")
print(f"🛒 주문당 평균 상품 종류: {len(order_items_df) / len(orders_df):.1f}개")


In [ ]:
# 🔍 데이터 관계 분석
print("🔍 데이터 관계 분석")
print("="*50)

# 📊 고객별 주문 분석
customer_orders = orders_df.groupby('customer_id').agg({
    'order_id': 'count',
    'final_amount': ['sum', 'mean']
}).round(2)

customer_orders.columns = ['주문_건수', '총_구매금액', '평균_주문금액']

print("👥 고객별 주문 패턴:")
print(f"평균 고객당 주문 건수: {customer_orders['주문_건수'].mean():.1f}건")
print(f"평균 고객당 총 구매금액: {customer_orders['총_구매금액'].mean():,.0f}원")
print(f"고객당 평균 주문금액: {customer_orders['평균_주문금액'].mean():,.0f}원")
print()

# 📦 카테고리별 매출 분석
category_sales = order_items_df.merge(products_df[['product_id', 'category']], on='product_id') \
                               .groupby('category')['total_price'].sum().sort_values(ascending=False)

print("🏷️ 카테고리별 매출 순위:")
for i, (category, sales) in enumerate(category_sales.head().items(), 1):
    print(f"{i}. {category}: {sales:,.0f}원 ({sales/category_sales.sum()*100:.1f}%)")
print()

# 🗺️ 지역별 주문 패턴 분석
region_analysis = orders_df.merge(customers_df[['customer_id', 'region']], on='customer_id') \
                           .groupby('region').agg({
                               'order_id': 'count',
                               'final_amount': 'sum'
                           }).sort_values('final_amount', ascending=False)

print("🗺️ 지역별 주문 패턴:")
for region, data in region_analysis.iterrows():
    print(f"{region}: {data['order_id']}건, {data['final_amount']:,.0f}원")
print()

# 🎯 고객 세그먼트별 구매 행동
segment_analysis = orders_df.merge(customers_df[['customer_id', 'segment']], on='customer_id') \
                            .groupby('segment').agg({
                                'order_id': 'count',
                                'final_amount': ['sum', 'mean']
                            }).round(2)

segment_analysis.columns = ['주문_건수', '총_매출', '평균_주문금액']

print("🎯 고객 세그먼트별 구매 행동:")
for segment, data in segment_analysis.iterrows():
    print(f"{segment}: {data['주문_건수']}건, 총 {data['총_매출']:,.0f}원, 평균 {data['평균_주문금액']:,.0f}원")


In [ ]:
# 🔍 종합 데이터 관계 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🔍 E-commerce 데이터 종합 분석', fontsize=16, fontweight='bold')

# 1️⃣ 고객 세그먼트별 평균 주문금액
axes[0, 0].bar(segment_analysis.index, segment_analysis['평균_주문금액'], 
               color=['gold', 'silver', 'lightcoral', 'lightsteelblue'], alpha=0.8)
axes[0, 0].set_title('🎯 고객 세그먼트별 평균 주문금액')
axes[0, 0].set_ylabel('평균 주문금액 (원)')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2️⃣ 카테고리별 매출 비중
axes[0, 1].pie(category_sales.head().values, labels=category_sales.head().index, 
               autopct='%1.1f%%', startangle=90)
axes[0, 1].set_title('🏷️ 카테고리별 매출 비중 (Top 5)')

# 3️⃣ 지역별 총 매출
axes[1, 0].bar(region_analysis.index, region_analysis['final_amount']/1000, 
               color='lightgreen', alpha=0.7)
axes[1, 0].set_title('🗺️ 지역별 총 매출')
axes[1, 0].set_ylabel('총 매출 (천원)')
axes[1, 0].tick_params(axis='x', rotation=45)

# 4️⃣ 일별 매출 트렌드
daily_sales = orders_df.groupby(orders_df['order_date'].dt.date)['final_amount'].sum()
axes[1, 1].plot(daily_sales.index, daily_sales.values/1000, 
                color='steelblue', linewidth=2, marker='o', markersize=3)
axes[1, 1].set_title('📈 일별 매출 트렌드')
axes[1, 1].set_ylabel('일별 매출 (천원)')
axes[1, 1].tick_params(axis='x', rotation=45)

# 💄 레이아웃 조정
plt.tight_layout()
plt.show()

# 📊 핵심 성과 지표 (KPI) 요약
print("\n📊 핵심 성과 지표 (KPI) 요약")
print("="*50)

# 전체 매출 지표
total_revenue = orders_df['final_amount'].sum()
total_orders = len(orders_df)
avg_order_value = orders_df['final_amount'].mean()
total_customers = customers_df['customer_id'].nunique()

print(f"💰 총 매출: {total_revenue:,.0f}원")
print(f"🛒 총 주문 건수: {total_orders:,}건")
print(f"💳 평균 주문 금액 (AOV): {avg_order_value:,.0f}원")
print(f"👥 총 고객 수: {total_customers:,}명")
print(f"📊 고객당 평균 주문 건수: {total_orders/total_customers:.1f}건")
print(f"📊 고객당 평균 생애 가치 (CLV): {total_revenue/total_customers:,.0f}원")

# 상품 지표
total_products = len(products_df)
sold_products = order_items_df['product_id'].nunique()
product_sell_through = sold_products / total_products * 100

print(f"\n📦 총 상품 수: {total_products:,}개")
print(f"📈 판매된 상품 수: {sold_products:,}개")
print(f"📊 상품 판매율: {product_sell_through:.1f}%")

# 주문 상태별 통계
completed_orders = (orders_df['status'] == 'completed').sum()
completion_rate = completed_orders / total_orders * 100

print(f"\n✅ 완료된 주문: {completed_orders:,}건")
print(f"📊 주문 완료율: {completion_rate:.1f}%")


In [ ]:
# 💾 처리된 데이터 저장
import os

# 처리된 데이터 디렉토리 확인/생성
processed_dir = Path('data/processed')
processed_dir.mkdir(parents=True, exist_ok=True)

print("💾 처리된 데이터 저장 중...")

# 🔍 분석 결과 데이터프레임 생성
analysis_results = {
    'customer_summary': customer_orders,
    'category_sales': category_sales.to_frame('total_sales'),
    'region_analysis': region_analysis,
    'segment_analysis': segment_analysis
}

# 📊 요약 통계 저장
summary_stats = {
    'total_revenue': total_revenue,
    'total_orders': total_orders,
    'avg_order_value': avg_order_value,
    'total_customers': total_customers,
    'total_products': total_products,
    'sold_products': sold_products,
    'completion_rate': completion_rate
}

# CSV 파일로 저장
for name, df in analysis_results.items():
    file_path = processed_dir / f"{name}.csv"
    df.to_csv(file_path, index=True, encoding='utf-8-sig')
    print(f"✅ {name}.csv 저장 완료")

# 요약 통계를 JSON으로 저장
import json
summary_path = processed_dir / "summary_statistics.json"
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary_stats, f, ensure_ascii=False, indent=2)
print(f"✅ summary_statistics.json 저장 완료")

print(f"\n📁 저장 위치: {processed_dir.absolute()}")

# 저장된 파일 목록 확인
print("\n📄 저장된 분석 파일 목록:")
for file in processed_dir.glob("*"):
    file_size = file.stat().st_size / 1024  # KB
    print(f"  📄 {file.name}: {file_size:.1f} KB")
